In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import os
from urllib import parse
import numpy as np

네이버 영화 메인 페이지 1~10위 영화 상세페이지 주소 가져오기

In [3]:
def step1_get_detail_url() :
    # 접속할 페이지의 주소
    site = 'https://movie.naver.com/movie/running/current.nhn?order=reserve'

    # 접속한다.
    response = requests.get(site)   

    bs = BeautifulSoup(response.content, 'html.parser')
    #print(bs)

    # a 태그들을 가져온다.
    a_list = bs.select('.top_thumb_lst a')

    # href 속성을 가져온다.
    df = pd.DataFrame()
    for idx in range(10) :       # 상위 10개만 가져오기
        href = a_list[idx].get('href')
        
        # 가져온 href 속성의 주소를 분석한 객체를 생성한다.
        a1 = parse.urlparse(href)
        
        # 주소를 분석한 객체서 쿼리 스트링을 가져온다
        query_str = parse.parse_qs(a1.query)
        
        # 추출한 쿼리스트링 데이터에서 원하는 파라미터 데이터를 추출한다.
        code = query_str['code'][0]
        

        # print(href)
        df = df.append([[code]], ignore_index=True)

    df.columns = ['code']
    df.to_csv('movie_code_list.csv', index=False, encoding='utf-8-sig')
    print('주소 저장 완료')

In [3]:
step1_get_detail_url()

주소 저장 완료


평점 더보기 주소 스크랩

In [4]:
def step2_get_reple_href() :
    # 스크랩한 영화 코드를 불러온다.
    code_frame = pd.read_csv('movie_code_list.csv')
    code_list = code_frame['code'].tolist()

    # 테스트용
 #   code_list = ['156464', '109906']


    # 영화코드와 주소를 합쳐서 요청할 주소를 만든다.
    url_list = pd.DataFrame()
    for code in code_list:
        site = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=%s&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false' % code
        # print(site)
        url_list = url_list.append([[site]], ignore_index=True)

    url_list.columns = ['url']
    url_list.to_csv('movie_url_list.csv', index=False, encoding='utf-8-sig')
    print('저장완료')

In [5]:
step2_get_reple_href()

저장완료


예매순 1~10 영화의 1페이지부터 마지막페이지 까지 순회하면서 평점과 리뷰 데이터 가져와서 저장

In [5]:
def step3_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    # url = url_list[0]

    for url in url_list :
        print(url)
        # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
        response = requests.get(url)
        bs = BeautifulSoup(response.content, 'html.parser')
        # print(bs)

        # 총 페이지 수를 구한다.
        strong = bs.select('.total em')
        #1->0
        score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
        # print(score_total)
        pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
        if score_total % 10 > 0 :
            pageCnt += 1
        print(pageCnt)

        # 전체 페이지를 돌면서 140평 데이터를 가져온다.
        # 현재 페이지
        now_page = 1

        #pageCnt = 5        # 테스트로 5페이지까지만
        
        while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

            if os.path.exists('star_score.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                result_df.columns = ['text', 'score']
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig')
            else :                                               # 이미 파일이 있으면 결과를 더한다.
                result_df.to_csv('star_score.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1

    print('저장완료')

In [7]:
step3_get_reply_data()

https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=152691&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
4
1 / 4
2 / 4
3 / 4
4 / 4
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=191633&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
36
1 / 36
2 / 36
3 / 36
4 / 36
5 / 36
6 / 36
7 / 36
8 / 36
9 / 36
10 / 36
11 / 36
12 / 36
13 / 36
14 / 36
15 / 36
16 / 36
17 / 36
18 / 36
19 / 36
20 / 36
21 / 36
22 / 36
23 / 36
24 / 36
25 / 36
26 / 36
27 / 36
28 / 36
29 / 36
30 / 36
31 / 36
32 / 36
33 / 36
34 / 36
35 / 36
36 / 36
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=14448&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
229
1 / 229
2 / 229
3 / 229
4 / 229
5 / 229
6 / 229
7 / 229
8 / 229
9 / 229
10 / 229
11 / 229
12 / 229
13 / 229
14 / 229
15 / 229
16 / 229
17 / 

323 / 514
324 / 514
325 / 514
326 / 514
327 / 514
328 / 514
329 / 514
330 / 514
331 / 514
332 / 514
333 / 514
334 / 514
335 / 514
336 / 514
337 / 514
338 / 514
339 / 514
340 / 514
341 / 514
342 / 514
343 / 514
344 / 514
345 / 514
346 / 514
347 / 514
348 / 514
349 / 514
350 / 514
351 / 514
352 / 514
353 / 514
354 / 514
355 / 514
356 / 514
357 / 514
358 / 514
359 / 514
360 / 514
361 / 514
362 / 514
363 / 514
364 / 514
365 / 514
366 / 514
367 / 514
368 / 514
369 / 514
370 / 514
371 / 514
372 / 514
373 / 514
374 / 514
375 / 514
376 / 514
377 / 514
378 / 514
379 / 514
380 / 514
381 / 514
382 / 514
383 / 514
384 / 514
385 / 514
386 / 514
387 / 514
388 / 514
389 / 514
390 / 514
391 / 514
392 / 514
393 / 514
394 / 514
395 / 514
396 / 514
397 / 514
398 / 514
399 / 514
400 / 514
401 / 514
402 / 514
403 / 514
404 / 514
405 / 514
406 / 514
407 / 514
408 / 514
409 / 514
410 / 514
411 / 514
412 / 514
413 / 514
414 / 514
415 / 514
416 / 514
417 / 514
418 / 514
419 / 514
420 / 514
421 / 514
422 / 514


In [6]:
# 140자평 데이터 전처리 함수
def text_preprocessing(text) :
    if text.startswith('관람객') :
        return text[3:]
    ######
    elif text.startswith('스포일러가 포함된 감상평입니다. 감상평 보기'):
        return text[23:]
    #####
    else :
        return text

# 평점 전처리 함수
def star_preprocessing(text) :
    value = int(text)

    if value <= 5 :
        return '0'
    else :
        return '1'

In [3]:
def step4_data_preprocessing() :
    # 수집한 데이터를 읽어온다.
    df = pd.read_csv('star_score.csv')

    # 전처리를 수행한다.
    df['text'] = df['text'].apply(text_preprocessing)
    df['score'] = df['score'].apply(star_preprocessing)

    # 학습데이터와 테스트 데이터로 나눈다.
    text_list = df['text'].tolist()
    star_list = df['score'].tolist()

    from sklearn.model_selection import train_test_split

    # 70%는 학습, 30%는 test
    text_train, text_test, star_train, star_test = train_test_split(text_list, star_list, test_size=0.3, random_state=0)

    return text_train, text_test, star_train, star_test


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import pickle
from konlpy.tag import *

In [10]:
step4_data_preprocessing()

(['\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t중국영화의 선입견을 깨트려준 영화. "최고다"라고 말할 수밖에 없음ㅠㅠ \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
  '\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t영상미랑 촬영이 엄청 돋보이는 영화였어요~ 몰입도가 엄청나고 기대했던만큼 좋은 영화였어요!! \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
  '\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t감히 장국영과 그 누구를 비교하지도 말라 \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n',
  '\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t

In [4]:
# 형태소 분석을 위한 함수
def tokenizer(text) :
    okt = Okt()
    return okt.morphs(text)

In [9]:
def step5_learning(X_train, y_train, X_test, y_test):
    # 주어진 데이터를 단어 사전으로 만들고 각 단어의 빈도수를 계산한 후 벡터화 하는 객체 생성
    tfidf = TfidfVectorizer(lowercase=False, tokenizer=tokenizer, ngram_range=(1,2)) #ngram (1,2)
     
    # 문장별 나오는 단어수 세서 수치화, 벡터화해서 학습을 시킨다.
    logistic = LogisticRegression(C=10.0, penalty='l2', random_state=0)

    pipe = Pipeline([('vect', tfidf), ('clf', logistic)])

    # 학습한다.
    pipe.fit(X_train, y_train)

    # 학습 정확도 측정
    y_pred = pipe.predict(X_test)
    print(accuracy_score(y_test, y_pred))

    # 학습된 모델을 저장한다.
    with open('pipe.dat', 'wb') as fp :
        pickle.dump(pipe, fp)
        
    print('저장완료')

In [10]:
def step6_using_model() :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np

    while True :
        text = input('리뷰를 작성해주세요 :')

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print('긍정적인 리뷰')
        else :
            print('부정적인 리뷰')

        print('정확도 : %.3f' % r1)

In [5]:
def one_movie_get_reply_data() :
    # csv에 저장되어 있는 url 데이터를 가져온다.
    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
    #    print(url)
    url = url_list[0]

       
    print(url)
    # 해당 영화의 첫 페이지 html 데이터를 가져온다. (총 몇건의 리뷰가 있는지 확인해서 "페이지수"를 계산하기 위함)
    response = requests.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    # print(bs)

    # 총 페이지 수를 구한다.
    strong = bs.select('.total em')
    #1->0
    score_total = int(strong[0].text.replace(',', ''))    # 쉼표 없애기 / int(정수형)로 만들기
    # print(score_total)
    pageCnt = score_total // 10     # 한페이지당 10개의 리뷰가 있어서
    if score_total % 10 > 0 :
        pageCnt += 1
    print(pageCnt)

    # 전체 페이지를 돌면서 140평 데이터를 가져온다.
    # 현재 페이지
    now_page = 1

#          pageCnt = 5        # 일단 테스트로 5페이지까지만 해보자.

    while now_page <= pageCnt :
            #sleep(1)
            #sleep(0.0001)
            # 요청할 페이지의 주소
            url2 = url + '&page=' + str(now_page)
            # print(url2)

            # 140자평 데이터를 추출한다.
            response2 = requests.get(url2)
            bs2 = BeautifulSoup(response2.content, 'html.parser')

            result_df = pd.DataFrame()

            # li 태그들을 가져온다.(score_reple 태그-리뷰-를 포함하고 있는)
            lis = bs2.select('.score_result li')

            for obj in lis :
                # 평점
                star_score = obj.select('.star_score em')[0].text
                # 140자평
                score_reple = obj.select('.score_reple p')[0].text
                # print(star_score)
                # print(score_reple)

                # 저장한다.
                result_df = result_df.append([[score_reple, star_score]], ignore_index=True)

                if os.path.exists('1917.csv') == False :     # 아직 파일이 없으면 파일을 만든다.
                    result_df.columns = ['text', 'score']
                    result_df.to_csv('1917.csv', index=False, encoding='utf-8-sig')
                else :                                               # 이미 파일이 있으면 결과를 더한다.
                    result_df.to_csv('1917.csv', index=False, encoding='utf-8-sig', mode='a', header=False)

            print("%d / %d" % (now_page, pageCnt))               # 진행경과를 보여준다. n번째 중 몇 번째 진행중인지.
            now_page += 1


    print('저장완료')

In [12]:
# 스크래핑 함수
def scrapping() :

    # 각 영화 코드 데이터를 가져와 저장한다.
    step1_get_detail_url()

    # 140자 평 데이터가 있는 페이지의 주소를 저장한다.
    step2_get_reple_href()

    # 140평 데이터를 가져온다.
    step3_get_reply_data()

# 학습 함수
def learing() :
    text_train, text_test, star_train, star_test = step4_data_preprocessing()
    step5_learning(text_train, star_train, text_test, star_test)

# 사용 함수
def using() :
    step6_using_model()

In [16]:
scrapping()

주소 저장 완료
저장완료
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=152691&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
4
1 / 4
2 / 4
3 / 4
4 / 4
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=191633&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
36
1 / 36
2 / 36
3 / 36
4 / 36
5 / 36
6 / 36
7 / 36
8 / 36
9 / 36
10 / 36
11 / 36
12 / 36
13 / 36
14 / 36
15 / 36
16 / 36
17 / 36
18 / 36
19 / 36
20 / 36
21 / 36
22 / 36
23 / 36
24 / 36
25 / 36
26 / 36
27 / 36
28 / 36
29 / 36
30 / 36
31 / 36
32 / 36
33 / 36
34 / 36
35 / 36
36 / 36
https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=14448&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false
229
1 / 229
2 / 229
3 / 229
4 / 229
5 / 229
6 / 229
7 / 229
8 / 229
9 / 229
10 / 229
11 / 229
12 / 229
13 / 229
14 / 229
15 / 229


321 / 514
322 / 514
323 / 514
324 / 514
325 / 514
326 / 514
327 / 514
328 / 514
329 / 514
330 / 514
331 / 514
332 / 514
333 / 514
334 / 514
335 / 514
336 / 514
337 / 514
338 / 514
339 / 514
340 / 514
341 / 514
342 / 514
343 / 514
344 / 514
345 / 514
346 / 514
347 / 514
348 / 514
349 / 514
350 / 514
351 / 514
352 / 514
353 / 514
354 / 514
355 / 514
356 / 514
357 / 514
358 / 514
359 / 514
360 / 514
361 / 514
362 / 514
363 / 514
364 / 514
365 / 514
366 / 514
367 / 514
368 / 514
369 / 514
370 / 514
371 / 514
372 / 514
373 / 514
374 / 514
375 / 514
376 / 514
377 / 514
378 / 514
379 / 514
380 / 514
381 / 514
382 / 514
383 / 514
384 / 514
385 / 514
386 / 514
387 / 514
388 / 514
389 / 514
390 / 514
391 / 514
392 / 514
393 / 514
394 / 514
395 / 514
396 / 514
397 / 514
398 / 514
399 / 514
400 / 514
401 / 514
402 / 514
403 / 514
404 / 514
405 / 514
406 / 514
407 / 514
408 / 514
409 / 514
410 / 514
411 / 514
412 / 514
413 / 514
414 / 514
415 / 514
416 / 514
417 / 514
418 / 514
419 / 514
420 / 514


In [13]:
import pandas as pd

In [ ]:
# using()

In [20]:
# scrapping()
learing()
using()

C:\Users\GiWoong\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.9757442116868799
저장완료
리뷰를 작성해주세요 :df


KeyboardInterrupt: 

In [9]:
def get_data(url,re_list):
    rslist = []
    count=0
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    for li in lis:
        rslist= {'reple' : li.p.text, 'score': li.find('em').getText()}
        re_list.append(rslist)
        
        
        #watch_movie = li.find('span', {'class':'ico_viewer'})

전처리 조사버리기

In [6]:
def one_movie_prepro(re_list): 
    import numpy as np
    df=pd.DataFrame(re_list)
    df.columns = ["리뷰","평점"]

    df['리뷰']=df['리뷰'].str.replace(pat='관람객', repl='', regex=False)
    df[df['리뷰'].str.contains("스포일러가 포함된 감상평입니다. 감상평 보기")]= np.nan #nan으로 변경

    df['리뷰']= df['리뷰'].str.strip()  # 앞 뒤 공백을 제거
    df=df.replace('',np.NaN)
    df=df.dropna()    
#     df.dropna()
    return df

예매 1순위 가져옴

In [10]:
def one_movie_get_review():
    #예매순위 1순위

    df = pd.read_csv('movie_url_list.csv')
    url_list = df['url'].tolist()

    # for url in url_list :
        #    print(url)
    test_url = url_list[0]
    resp = requests.get(test_url)
    html = BeautifulSoup(resp.content, 'html.parser')
    result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[0].getText()
    total_count = int(result.replace(',', ''))
    re_list= []


    for i in range(1, int(total_count / 10) + 1):
    #for i in range(1, int(total_count / 100) + 1):
        url = test_url + '&page=' + str(i)
        #print('url: "' + url + '" is parsing....')
        get_data(url,re_list)
    review_df = pd.DataFrame()
    review_df=one_movie_prepro(re_list)

    return review_df
#     re_list


In [11]:
df=one_movie_get_review()


In [12]:
df.tail(60)

,리뷰,평점
263,스토리는 괭장히 단순하지만 영화 시작부터 끝까지 긴장이 되는 연기와 연출은 정말 재...,9
265,실화와 역사가 녹아 마음 깊이 뜨거워진다.,10
266,나까지 심장이 두근거린다 ㅋㅋ,7
267,교도관 츤데레.. ..ㅋㅋㄱㄱ,8
268,긴장감 속 숨은 교훈.평등을 위한 그들의 투쟁 respect.,10
269,코로나 시대에 방탈출 하는 기분!,10
270,12세라고 만만히 봤다가 내내 긴장하면서 봤음 ㅎㅎ?? 긴장감 있게 잘봤습니다,9
271,쫄깃쫄깃 우동면발 같은 실화극!,8
273,조마조마한 느낌을 오랜만에 받았다. 당시 사회상을 조금이나마 알 수 있었던 의미있는...,10
274,긴장감 넘치는 탈옥의 과정,7


In [13]:
def step7_insert_review(df) :
    # 객체를 복원한다.
    with open('pipe.dat', 'rb') as fp:
        pipe = pickle.load(fp)

    import numpy as np
    import pandas as pd

    
    rlist=df["리뷰"].tolist()
    df_result = pd.DataFrame()
    
    for i in rlist :
        text = i

        str = [text]
        # 예측 정확도
        r1 = np.max(pipe.predict_proba(str) * 100)
        # 예측 결과
        r2 = pipe.predict(str)[0]

        if r2 == '1' :
            print(text)
            
            print('긍정적인 리뷰')
            df_result = df_result.append([[text, r1, r2]],ignore_index=True)
            print('----------------------------')
        else :
            print(text)
            print('부정적인 리뷰')
            df_result = df_result.append([[text, r1, r2]],ignore_index=True)
            print('----------------------------')

        print('정확도 : %.3f' % r1)
    
    df_result.columns = ['리뷰','정확도','결과']
    return df_result


In [14]:
final_result = step7_insert_review(df)

이것은... 사진 세울 때 쓰는 겁니다ㅎ
긍정적인 리뷰
----------------------------
정확도 : 95.861
영화 재밌습니다. 다니엘 래드클리프 완전 열연이네요. 열쇠맞출때마다 몰입감과 긴장감이 공존하는데 꿀잼입니다. 올해 본 영화 중 탑 3에 들어감. 저게 실화라는게 더 놀랍네요.
긍정적인 리뷰
----------------------------
정확도 : 99.708
코로나 탈출 방탈출 같이 갇힌 공간을 탈출하고 싶은 그 맘이 커서 그런지 재밌게 봤음 방탈출카페 가고 싶다
긍정적인 리뷰
----------------------------
정확도 : 98.916
다니엘 래드클리프.. 이제는  배우로 도약하는거같다
긍정적인 리뷰
----------------------------
정확도 : 97.575
심장 쫄리는 수준이 장난 아님. 말도 안 되는 것 같은데 실화라니 더 충격
긍정적인 리뷰
----------------------------
정확도 : 97.623
몰입도라든가 연출이좋음
긍정적인 리뷰
----------------------------
정확도 : 98.318
나라면 그냥 12년 살았을듯....쫄려 죽는줄..
긍정적인 리뷰
----------------------------
정확도 : 89.424
껌으로 열쇠 붙일때가 젤 긴장감 있었다ㅋㅋㅋ 스토리는 실화기반이라 독특한 느낌은 없었다. 쇼생크 탈출류의 감옥이야기
긍정적인 리뷰
----------------------------
정확도 : 97.764
긴장감,몰입감, 통쾌함을 느낀  재미난영화
긍정적인 리뷰
----------------------------
정확도 : 97.931
그동안 봐온 감옥탈출 영화중 가장 소박했지만 열쇠라는 작은 매체 하나만으로 이렇게 스릴을 유도하는 경우는 개인적으로 처음 봐서 실화이지만서도 참신하게 다가온 작품이었습니다.
긍정적인 리뷰
----------------------------
정확도 : 97.633
진짜 너무 재밌음 제빌 봐주세요 부탁임

심장 쫄깃해짐... 몰입감 대박 ㅎㅎ 연기도 잘하규 마지막에 브금 짱좋음
긍정적인 리뷰
----------------------------
정확도 : 99.970
너무 재밌었어요ㅠㅠ 진짜 간만에 영화다운영화가 나왔네요..이시기에 나온게 아쉽,,.
긍정적인 리뷰
----------------------------
정확도 : 98.941
다니엘 해리포터랑 오버랩 한번도 안될만큼 연기력 미쳐돌아가셨고 실화바탕이라 자칫하면 그저 흘러가는대로 지나갔을 내용이 연출미쳐서 보는내내 지릴뻔 했습니다. 제발봐주세요 진짜 아 진짜 아직도 긴장감이 안가셤ㅎ어요
긍정적인 리뷰
----------------------------
정확도 : 92.615
스릴넘치는 긴장감 숨도못쉬어요
긍정적인 리뷰
----------------------------
정확도 : 97.320
생각보다 흥미진진 손에땀을쥐게함
긍정적인 리뷰
----------------------------
정확도 : 97.269
와 무슨 공포영화 보는것처럼엄청 쪼입니다 긴장감 대박
긍정적인 리뷰
----------------------------
정확도 : 93.099
제가 인생 통틀어서 실화영화중에 제일 재밌게 봤어요...아무래도 실화이다보니 영화중간중간 개연성이 떨어지지만 그래도 나름 긴장의 끈은 초반부터 엔딩크레딧까지 이어지는게 이 영화의 묘미입니다. 배우들 연기는 말할것도 ...
긍정적인 리뷰
----------------------------
정확도 : 98.089
재밌다. 실화는 늘 재밌다. 너무 멋있는 사람들의 이야기. 마치 내가 감옥에 들어가 있는 듯한 기분을 영화를 통해 느꼇고 공포영화에서 주는기분나쁜 긴장감이 아닌 어떤 중요한 일을 이루어 내기 위해 생기는 긴장감을 주...
긍정적인 리뷰
----------------------------
정확도 : 98.812
진짜 최고 !!그리고 감동. 기대 훨 이상이었고 지루할 틈 없었음
긍정적인 리뷰
----------------------------
정확도 : 9

----------------------------
정확도 : 99.642
마지막 순간에 저도 감격스러워서 울었네요 1917에 비교하긴 그렇지만 저에게는 진짜 감동적인 영화였어요 영상미랑 음악이랑 연기력이 와..꼭 보시는걸 추천드립니다
긍정적인 리뷰
----------------------------
정확도 : 99.433
전만족하며봤어요 탈출특유의 긴장감도 너무좋았고 내용도 좋았어요 ㅎ
긍정적인 리뷰
----------------------------
정확도 : 97.436
실화에충격..쇼생크탈출의 업그레이드버젼
긍정적인 리뷰
----------------------------
정확도 : 96.985
익스펙토 페트로눔..
긍정적인 리뷰
----------------------------
정확도 : 91.462
심장 쫄리면서 봤네요 추천
긍정적인 리뷰
----------------------------
정확도 : 98.705
긴장감 미띰 :; 와 아주 ^^ㅏ발적임 너무 재밌음 꼭 보셈 와..
긍정적인 리뷰
----------------------------
정확도 : 99.073
워낙볼게없으니 이게 볼만하다고 느껴질뿐. 마지막 십오분 제외하면 개지루.
긍정적인 리뷰
----------------------------
정확도 : 84.062
코스 요리를 시켰는데, 정작 메인디쉬는 안 나옴.
긍정적인 리뷰
----------------------------
정확도 : 70.146
쇼생크탈출이 선녀라면, 이 영화는 하녀다.긴장은 있으나 감동이 없고목적은 있으나 꿈이 없고사실은 있으나 사연이 없다.
긍정적인 리뷰
----------------------------
정확도 : 93.543
정말 쫄깃쫄깃하다
긍정적인 리뷰
----------------------------
정확도 : 96.540
어짜파 탈출 성공할꺼 아는데도 조마조마하게 만든다
긍정적인 리뷰
----------------------------
정확도 : 95.545
뭔가 몹시 평범한 영화임 긴장감 크게없

----------------------------
정확도 : 94.127
배경이 옛날이라 그런지 보안이 지나치게 허술하다는 생각에 좀 김빠지네요. 그래도 긴장감이나 갈등이 적절하고 미사여구 없이 담담하게 담아낸 것 같습니다.
긍정적인 리뷰
----------------------------
정확도 : 86.768
영화 보는내내 긴장감과 몰입도가 쩔었다.. 돈 아깝지 않은 영화..
긍정적인 리뷰
----------------------------
정확도 : 96.390
다니엘 나오는 영환데 못봐ㅜㅜㅜ 아니 대체 왜 코로나로 난리일 때 개봉하는 거에요ㅜㅜ 코로나 아니었어도 고삼이라 못보지만ㅎㅋㅎㅋ 안 봐도 잘 했을테니 미리 평점 누르고 갑니다 나중에 볼게요!
긍정적인 리뷰
----------------------------
정확도 : 95.666
보고 있으면 해리포터에 민경훈 오버랩돼요 영화는 재밌습니다
긍정적인 리뷰
----------------------------
정확도 : 86.220
나무열쇠로 감방문을 열다니... 역시 마법사!!!
긍정적인 리뷰
----------------------------
정확도 : 95.515
탈옥할때 긴장감은 두고나왔네
부정적인 리뷰
----------------------------
정확도 : 52.559
밑져야 본전 아니 땐 굴뚝에 연기나랴
부정적인 리뷰
----------------------------
정확도 : 62.315
들킬까봐 조마조마한 장면들은 꽤 있다. 딱히 재밌진 않지만 시간떼우기엔 무난함.
긍정적인 리뷰
----------------------------
정확도 : 93.592
그냥 무난함 이게 4점 짜리가 된다고 그럼 우리나라 2점 짜리 투성인데 알바들아 ㅋㅋㅋㅋ
긍정적인 리뷰
----------------------------
정확도 : 68.537
심장은 개 쫄림 근데 너무 루즈하고 실화라 어쩔 수 없지만 결말도 너무 직관적임
긍정적인 리뷰
----------------------------
정확도

In [15]:
final_result.tail(50)  #1 긍정, 0 부정

,리뷰,정확도,결과
252,뻔한 스토리와 실화라서 결말이 예상되는 부분이 있음에도 긴장감 조절이 수준급이라 그...,89.372235,1
253,애초에 실화바탕으로 제작한거라 재미를 추구하기보다는 전체적으로 긴장감과 배우들 연기...,97.807998,1
254,"남아공 아파르트헤이트 시절을 배경으로 하는 영화, 긴장감있는 감옥 연출은 높게 평가...",94.740835,1
255,특별하게 자극적인 장면도 없이 이렇게 몰입감 있는 탈출 영화는 오랜만..,98.446507,1
256,스릴있어서 너무 좋았는데 맘놓고 보기는 힘들었어요중간에 하 ㅅㅂ 언제탈출하냐 싶을정...,91.207211,1
257,와나 18 그렇게 애써 다 만들 필요없이 밤에는 건강도 안좋아보이는 할배 돼지 한마...,95.277951,1
258,6점정도. 킬링타임용.,80.243855,1
259,해리포터와 프리토리아의 죄수,96.672118,1
260,"음,,,,타임킬링 약간 이하7점주고싶으나 고평가가 너무 많아 밸런스로 6점 드림",90.819207,1
261,큰 기대없이 본 영화인데 연출력이 딸려서 무지 아쉽긴했네요 해리포터가 연기를 못한건...,95.893957,1


In [27]:
review=final_result['리뷰'].tolist()
accuracy=final_result['정확도'].tolist()
result=final_result['결과'].tolist()

[95.8608363843271,
 99.70811951271817,
 98.9156496960925,
 97.57459040649039,
 97.62341298535156,
 98.31810979002876,
 89.42375122831987,
 97.7639187292885,
 97.93065274440139,
 97.63268056544209,
 97.41832673768478,
 89.52481613040959,
 93.6839285262317,
 99.92435239017041,
 97.10881589857108,
 91.42854840064581,
 91.62204920276078,
 96.672247063277,
 99.34885728384835,
 96.8975477541,
 98.73137272649458,
 98.60718735645189,
 94.00151492190827,
 99.47699751943652,
 98.15095616615635,
 97.58004456114413,
 98.87092564536972,
 98.70905129020788,
 97.72781647853412,
 99.52367097175704,
 96.58653500509375,
 95.8860996354268,
 97.04269561703516,
 68.56520552606007,
 95.9895358798021,
 95.16912928976248,
 95.2664317909483,
 95.83392262030799,
 99.54609468389395,
 99.9890265850295,
 91.87416181163246,
 99.30162361916497,
 95.79001944728282,
 98.91565240039714,
 89.57698970300079,
 99.4621768534276,
 94.48895666252196,
 95.19837796087376,
 96.6462297699853,
 97.12005782299902,
 98.762728167825